In [3]:
import cv2
import numpy
import imageio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os


In [4]:
#obtener el width y height maximo de las imagenes a preparar
def getMaxSize(folderUrl):
  moreHeight = 0
  moreWidht = 0
  myListImages = os.listdir(folderUrl)
  for cl in myListImages: 
    img = cv2.imread(folderUrl + cl)
    old_image_height, old_image_width, channels = img.shape

    if old_image_height > moreHeight:
      moreHeight = old_image_height

    if old_image_width > moreWidht:
      moreWidht = old_image_width

  return moreHeight, moreWidht

In [10]:
#añadir padding a la imagen original con las nuevas medidas
def addPaddingToImage(imageName, new_height, new_width): 
  img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/US_ROI/' + imageName) 
  delta_w = new_width - img.shape[0]
  delta_h = new_height - img.shape[1]
  top, bottom = delta_h // 2, delta_h - (delta_h // 2)  
  left, right = delta_w // 2, delta_w - (delta_w // 2) 
  image= np.pad(img, ((top, bottom), (left, right), (0,0)), mode='constant')
  
  #Save image with padding
  urlToSave = '/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/Padding/Padding1/' + imageName  
  cv2.imwrite(urlToSave, image)
  #Initial Resize
  img = Image.open(urlToSave)
  imgResized = img.resize((500, 500))
  urlToSave2 = '/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/Padding/PaddingReady/' + imageName
  imgResized.save(urlToSave2)  

In [11]:
from PIL import Image

#resize la imagen original con las nuevas medidas 
def resizeImage(imageName): 
  basewidth = 224
  img = Image.open('/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/Padding/PaddingReady/' + imageName )
  wpercent = (basewidth / float(img.size[0]))
  hsize = int((float(img.size[1]) * float(wpercent)))
  img = img.resize((basewidth, hsize), Image.ANTIALIAS)
  img.save('/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/Resize/{}'.format(imageName))

In [12]:
def prepareROIImages():
  new_height = 0
  new_width = 0  

  moreHeight, moreWidht = getMaxSize('/content/drive/MyDrive/Colab Notebooks/US_ROI/')

  if moreHeight < 500 and moreWidht < 500:
    new_height = 500
    new_width = 500
  
  myListImages = os.listdir('/content/drive/MyDrive/Colab Notebooks/US_ROI/')
  for cl in myListImages: 
    addPaddingToImage(cl, new_height, new_width)
    resizeImage(cl)

In [13]:
prepareROIImages()

In [47]:
#Read csv
import pandas as pd
dataframe = pd.read_csv(r"DatasetB-csv.csv",sep=',',header=None)
dataframe.head(10)
#dataframe.shape[0]

,0,1
0,ROI_0000001,Benign
1,ROI_0000002,Benign
2,ROI_0000003,Benign
3,ROI_0000004,Benign
4,ROI_0000005,Benign
5,ROI_0000006,Benign
6,ROI_0000007,Benign
7,ROI_0000008,Benign
8,ROI_0000009,Benign
9,ROI_0000010,Benign


In [48]:
#DatasetByClass
for index, row in dataframe.iterrows():
  imageName = row[0]
  classif = row[1]
  imageName += '.png'

  #Fix image names
  if imageName == 'ROI_0000298.png':
    imageName = 'ROI_000298.png'
  if imageName == 'ROI_0000304.png':
    imageName = 'ROI_000304.png'
  

  img = Image.open('/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/Resize/' + imageName )

  urlToSaveClass = '/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/DatasetByClass'
  if classif == 'Benign':
    urlToSaveClass += '/Benign'  
  elif classif == 'Malignant':
    urlToSaveClass += '/Malignant' 
  
  urlToSaveClass += "/" + imageName
  #print(urlToSaveClass)
  img.save(urlToSaveClass)
 
 

In [51]:
#Split dataset
myImagesMalignant = os.listdir('/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/DatasetByClass/Malignant/')
myImagesBenign = os.listdir('/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared_ZeroPadding/DatasetByClass/Benign/')
print('malignant:')
print(len(myImagesMalignant))
print('benign:')
print(len(myImagesBenign))

malignant:
54
benign:
109
